
<a href="https://colab.research.google.com/github/takzen/pytorch-black-belt/blob/main/18_Num_Workers_and_Pin_Memory.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# 🥋 Lekcja 18: Optymalizacja Transferu Danych (num_workers & pin_memory)

Trening sieci to sztafeta.
1.  **CPU (Dysk/RAM):** Czyta plik -> Dekoduje JPG -> Augmentacja -> Tensor.
2.  **Transfer:** Kopiowanie z RAM do VRAM.
3.  **GPU:** Obliczenia (Forward/Backward).

Jeśli krok 1 i 2 są wolniejsze niż 3, Twoje drogocenne GPU leży odłogiem.

**Rozwiązania:**
1.  **`num_workers > 0`**:
    *   Domyślnie (`0`) główny proces robi wszystko: Wczytaj -> Trenuj -> Wczytaj.
    *   Z workerami (`4`): Workery ładują kolejkę w tle. Główny proces tylko bierze gotowe.
2.  **`pin_memory=True`**:
    *   Pamięć RAM dzieli się na **Pageable** (zwykła) i **Pinned** (sztywna).
    *   Transfer: Pageable RAM -> Pinned RAM -> GPU VRAM.
    *   Ustawiając `pin_memory=True`, wrzucamy dane od razu do Pinned RAM. Oszczędzamy jedno kopiowanie CPU-CPU.

In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import time

# Sprawdźmy sprzęt
if torch.cuda.is_available():
    device = "cuda"
    print("✅ Mamy GPU! pin_memory ma sens.")
else:
    device = "cpu"
    print("⚠️ Brak GPU. pin_memory nic nie da, ale num_workers nadal działa.")

# Generujemy spory dataset (100MB)
data_size = 10000
features = 1000
dataset = TensorDataset(torch.randn(data_size, features), torch.randn(data_size, 1))

print(f"Dataset gotowy: {data_size} próbek.")

✅ Mamy GPU! pin_memory ma sens.
Dataset gotowy: 10000 próbek.


## 1. Pin Memory (Autostrada)

Standardowy tensor w PyTorch żyje w pamięci stronicowanej (Pageable). System operacyjny może go przesuwać lub zrzucić na dysk (Swap).
Karta graficzna (DMA - Direct Memory Access) nie może czytać z takiej pamięci. Wymaga pamięci "przypiętej" (Pinned), która fizycznie nie zmienia adresu.

Ustawienie `pin_memory=True` w DataLoaderze sprawia, że PyTorch alokuje specjalny bufor w RAM, co przyspiesza transfer `to(device)` nawet o **2-3 razy**.

In [2]:
# Tworzymy tensor w zwykłym RAM
x = torch.randn(5, 5)
print(f"Czy jest przypięty? {x.is_pinned()}")

# Przypinamy go ręcznie (to robi DataLoader pod spodem)
x_pinned = x.pin_memory()
print(f"Czy teraz jest przypięty? {x_pinned.is_pinned()}")

# Benchmark transferu (tylko jeśli masz GPU)
if device == "cuda":
    # 1. Bez Pinningu
    start = time.time()
    for _ in range(100):
        _ = x.to(device)
    print(f"Czas Pageable -> GPU: {time.time() - start:.4f}s")
    
    # 2. Z Pinningiem
    start = time.time()
    for _ in range(100):
        _ = x_pinned.to(device, non_blocking=True) # non_blocking pozwala na asynchroniczność!
    print(f"Czas Pinned -> GPU:   {time.time() - start:.4f}s")
    print("(Przy małych tensorach różnica jest mała, przy Gigabajtach - ogromna).")

Czy jest przypięty? False
Czy teraz jest przypięty? True
Czas Pageable -> GPU: 0.0065s
Czas Pinned -> GPU:   0.0010s
(Przy małych tensorach różnica jest mała, przy Gigabajtach - ogromna).


## 2. Num Workers (Wielowątkowość)

To parametr, który decyduje, ile podprocesów "przygotowuje posiłki" dla GPU.

**Zasada kciuka:**
*   `0`: Debugowanie (najbezpieczniej, działa na Windows w Jupyterze).
*   `2-4`: Zazwyczaj optymalne.
*   `>8`: Często spowalnia (zbyt duży narzut na zarządzanie procesami).

**⚠️ UWAGA DLA UŻYTKOWNIKÓW WINDOWS:**
Jesteśmy w Jupyter Notebook na Windows. Ustawienie `num_workers > 0` tutaj często powoduje błędy (BrokenPipeError, RuntimeError), o których rozmawialiśmy wcześniej.
Dlatego w poniższym teście **zostawimy 0**, ale kod jest gotowy, by zmienić tę liczbę na serwerze Linuxowym.

In [3]:
def benchmark_loader(num_workers, pin_memory):
    # Tworzymy loader z zadanymi parametrami
    # UWAGA: Na Windows w Jupyterze num_workers > 0 może zawiesić kernel!
    # Jeśli to uruchamiasz u siebie, zachowaj ostrożność.
    loader = DataLoader(
        dataset, 
        batch_size=128, 
        num_workers=num_workers, 
        pin_memory=pin_memory,
        shuffle=True
    )
    
    start = time.time()
    for batch_x, batch_y in loader:
        # Symulacja transferu na GPU
        if device == "cuda":
            batch_x = batch_x.to(device, non_blocking=pin_memory)
            batch_y = batch_y.to(device, non_blocking=pin_memory)
            
        # Symulacja obliczeń (krótka)
        _ = batch_x * 2 
        
    return time.time() - start

print("--- BENCHMARK (Symulacja) ---")

# Test 1: Baza (Jeden proces, zwykła pamięć) - Bezpieczne na Windows
t1 = benchmark_loader(num_workers=0, pin_memory=False)
print(f"Workers=0, Pin=False: {t1:.4f}s")

# Test 2: Tylko Pin Memory (Bezpieczne na Windows i GPU)
if device == "cuda":
    t2 = benchmark_loader(num_workers=0, pin_memory=True)
    print(f"Workers=0, Pin=True:  {t2:.4f}s")
    print(f"Zysk z samego Pinningu: {t1/t2:.2f}x")

# Test 3: Workers > 0 (RYZYKOWNE W NOTATNIKU NA WINDOWS - POMIJAMY)
# Na Linuxie/Produkcji ustawiłbyś tu np. num_workers=4
# t3 = benchmark_loader(num_workers=4, pin_memory=True) 
print("Workers=4: (Pominięto ze względu na stabilność kernela na Windows)")

--- BENCHMARK (Symulacja) ---
Workers=0, Pin=False: 0.1539s
Workers=0, Pin=True:  0.0782s
Zysk z samego Pinningu: 1.97x
Workers=4: (Pominięto ze względu na stabilność kernela na Windows)


## 🥋 Black Belt Summary

Jak konfigurować `DataLoader` na produkcji?

1.  **`pin_memory=True`**: ZAWSZE, jeśli używasz GPU. To darmowe przyspieszenie.
2.  **`num_workers`**:
    *   Zacznij od `4`.
    *   Jeśli masz szybki dysk (NVMe) i proste dane, CPU nie jest wąskim gardłem.
    *   Jeśli robisz ciężką augmentację w locie (obracanie, skalowanie), zwiększ liczbę workerów, żeby GPU nie czekało.
3.  **`non_blocking=True`**: Przy `x.to(device)` używaj tej flagi razem z `pin_memory`. Pozwala to GPU wykonywać obliczenia na poprzednim batchu w tym samym czasie, gdy CPU przesyła następny batch.